In [32]:
import requests
import pandas as pd
import numpy  as np

from random import *
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [3]:
log_data = pd.read_csv('./dataset.ver3/user_log_data.csv')

## dataset 만들기

## 전처리

In [4]:
# 1. userId가 USER999인 것을 지워줌 -> 로그인 후 사용하기로 해서 이 과정 쓸모 없어짐.
cleaned_log = log_data[log_data['userId'] != 'USER999']
cleaned_log.info() # 총 49,947행
cleaned_log.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   userId       100000 non-null  object
 1   contentcode  100000 non-null  int64 
 2   contentdes   100000 non-null  object
 3   click        100000 non-null  int64 
 4   date         100000 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.6+ MB


,contentcode,click
count,1.000000e+05,100000.000000
mean,1.362862e+06,1.996840
std,1.043626e+06,0.817103
min,1.252660e+05,1.000000
25%,1.346540e+05,1.000000
50%,1.496551e+06,2.000000
75%,2.569365e+06,3.000000
max,2.714351e+06,3.000000


In [6]:
item_lookup = log_data[['contentcode','contentdes']].drop_duplicates() # 중복값 처리(unique한 1개의 key만 남기고 나머지 중복은 제거) 
item_lookup['contentcode'] = item_lookup['contentcode']

In [7]:
# matrix가 매우 크기 때문에 Sparse matrix로 바꾸어주어서
# zero가 아닌 값들의 위치와 그 값만 저장하도록 메모리 절약!
cleaned_log = log_data[['userId','contentcode','click']]
grouped_cleaned = cleaned_log.groupby(['userId','contentcode']).sum().reset_index() #reset_index() 행 인덱스 초기화


# click한 애들만 뽑기 : 우리 데이터 특성상 무조건 1 이상 임.
grouped_purchased = grouped_cleaned[grouped_cleaned['click'] > 0]

In [12]:
customers = list(np.sort(grouped_purchased['userId'].unique()))
products = list (grouped_purchased['contentcode'].unique())
quantity = list(grouped_purchased['click'])

# cat.code: Return Series of codes as well as the index.
rows = grouped_purchased['userId'].astype('category').cat.codes 
cols = grouped_purchased['contentcode'].astype('category').cat.codes

# csr: Compressed Sparse matrix by Row
purchase_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape = (len(customers),len(products)))

In [14]:
# Sparsity: 얼마나 비어있나?
# sparse.nonzero(): Returns a tuple of arrays (row,col) containing the indices of the non-zero elements of the matrix.
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

# 95.06860937855105

99.58127779176105

In [17]:
import random

In [25]:
def make_train (matrix, percentage = .2):
    '''
    -----------------------------------------------------
    설명
    유저-아이템 행렬 (matrix)에서 
    1. 0이상의 값을 가지면 1의 값을 갖도록 binary하게 테스트 데이터를 만들고
    2. 훈련 데이터는 원본 행렬에서 percentage 비율만큼 0으로 바뀜
    
    -----------------------------------------------------
    반환
    training_set: 훈련 데이터에서 percentage 비율만큼 0으로 바뀐 행렬
    test_set:     원본 유저-아이템 행렬의 복사본
    user_inds:    훈련 데이터에서 0으로 바뀐 유저의 index
    '''
    test_set = matrix.copy()
    test_set[test_set !=0] = 1 # binary하게 만들기
    
    training_set = matrix.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    
    random.seed(0)
    num_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples)
    
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    # eliminate_zeros(): Remove zero entries from the matrix
    training_set[user_inds, item_inds] = 0
    training_set.eliminate_zeros()
    
    return training_set, test_set, list(set(user_inds))

In [19]:
product_train, product_test, product_users_altered = make_train(purchase_sparse, 0.2)

## ALS for Implicit Feedback

- 손코딩

In [28]:
RandomGenerator = np.random.RandomState(seed=0)
new_random_number = RandomGenerator.random(size=10)

In [29]:
conf = (40*product_train) # sparse 행렬 형태를 유지하기 위해서 1을 나중에 더함
#pd.DataFrame(conf.todense())
num_user = conf.shape[1]
num_user

23838

In [42]:
def implicit_weighted_ALS(training_set, lambda_val =.1, alpha = 40, n_iter = 10, rank_size = 20, seed = 0):
    '''
    협업 필터링에 기반한 ALS
    -----------------------------------------------------
    input
    1. training_set : m x n 행렬로, m은 유저 수, n은 아이템 수를 의미. csr 행렬 (희소 행렬) 형태여야 함 
    2. lambda_val: ALS의 정규화 term. 이 값을 늘리면 bias는 늘지만 분산은 감소. default값은 0.1
    3. alpha: 신뢰 행렬과 관련한 모수 (C_{ui} = 1 + alpha * r_{ui}). 이를 감소시키면 평점 간의 신뢰도의 다양성이 감소
    4. n_iter: 반복 횟수. 논문에서는 10~ 15회 정도 설정
    5. rank_size: 유저/ 아이템 특성 벡터의 잠재 특성의 개수. 논문에서는 20 ~ 200 사이를 추천하고 있음. 이를 늘리면 과적합 위험성이 있으나 
    bias가 감소
    6. seed: 난수 생성에 필요한 seed
    -----------------------------------------------------
    반환
    유저와 아이템에 대한 특성 벡터
    '''
    
    # 1. Confidence matrix
    # C = 1+ alpha * r_{ui}
    conf = (alpha*training_set) # sparse 행렬 형태를 유지하기 위해서 1을 나중에 더함
    
    num_user = conf.shape[0]
    num_item = conf.shape[1]

    # X와 Y 초기화
    rstate = np.random.RandomState(seed)
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size)))
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size)))
    # sparse.eye: Sparse matrix with ones on diagonal 
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    
    # 정규화 term: 𝝀I
    lambda_eye = lambda_val * sparse.eye(rank_size)
    
    # 반복 시작
    for i in range(n_iter):
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        
        # Y를 고정해놓고 X에 대해 반복
        # Xu = (yTy + yT(Cu-I)Y + 𝝀I)^{-1} yTCuPu
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Cu
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            # Cu-I: 위에서 conf에 1을 더하지 않았으니까 I를 빼지 않음 
            CuI = sparse.diags(conf_samp, [0])
            # yT(Cu-I)Y
            yTCuIY = Y.T.dot(CuI).dot(Y)
            # yTCuPu
            yTCupu = Y.T.dot(CuI+Y_eye).dot(pref.T)
            
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu)
        
        # X를 고정해놓고 Y에 대해 반복
        # Yi = (xTx + xT(Cu-I)X + 𝝀I)^{-1} xTCiPi
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray()
            pref = conf_samp.copy()
            pref[pref!=0] = 1
            
            #Ci-I
            CiI = sparse.diags(conf_samp, [0])
            # xT(Ci-I)X
            xTCiIX = X.T.dot(CiI).dot(X)
            # xTCiPi
            xTCiPi = X.T.dot(CiI+ X_eye).dot(pref.T)
            
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            
        return X, Y.T

In [43]:
user_vecs, item_vecs = implicit_weighted_ALS (product_train
                                              , lambda_val = 0.1
                                              , alpha = 15
                                              , n_iter = 10
                                              , rank_size = 20)

In [44]:
# 특정한 유저의 예측된 평점을 구하려면 유저 벡터와 아이템 벡터의 내적곱을 하면 됨.
first = user_vecs[0].dot(item_vecs).toarray() # 1x3650
first[0,:5]

array([-4.00744756e-05,  2.40477817e-05,  1.40552112e-05,  2.37901547e-05,
        3.03637466e-05])

## 추천 시스템 평가하기

In [45]:
from sklearn import metrics

In [46]:
def auc_score (test, predictions):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [47]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    가려진 정보가 있는 유저마다 AUC 평균을 구하는 함수
    ----------------------------------------
    input
    1. training_set: make_train 함수에서 만들어진 훈련 데이터 (일정 비율로 아이템 구매량이 0으로 가려진 데이터)
    2. prediction: implicit MF에서 나온 유저/아이템 별로 나온 예측 평점 행렬
    3. altered_users: make_train 함수에서 아이템 구매량이 0으로 가려진 유저
    4. test_set: make_train함수에서 만든 테스트 데이터
    ----------------------------------------
    반환
    추천시스템 유저의 평균 auc
    인기아이템 기반 유저 평균 auc
    '''
    # 리스트 초기화
    store_auc = []
    popularity_auc = []
    
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # 모든 유저의 아이템별 구매횟수 합
    item_vecs = predictions[1] # 아이템 latent 벡터
    
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1) # 유저의 훈련데이터
        zero_inds = np.where(training_row == 0) # 가려진 아이템 Index
        
        # 가려진 아이템에 대한 예측
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # 가려진 아이템에 대한 실제값
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        
        # 가려진 아이템에 대한 popularity (구매횟수 합)
        pop = pop_items[zero_inds]
        
        # AUC 계산 
        store_auc.append(auc_score(actual, pred))
        popularity_auc.append(auc_score(actual,pop))
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  

In [48]:
calc_mean_auc(product_train, product_users_altered, predictions, product_test)
# AUC for our recommender system

(0.494, 0.629)

## 추천 예시

In [49]:
def get_items_purchased(customer_id, mf_train, customer_list, products_list, item_lookup):
    '''
    특정 유저가 구매한 목록을 보여주는 함수
    ----------------------------------------
    INPUT
    1. customer_id: 고객 ID
    2. mf_train: 훈련 데이터 평점
    3. customers_list: 훈련 데이터에 쓰인 고객 목록
    4. products_list: 훈련 데이터에 쓰인 아이템 목록
    5. item_lookup: 유니크한 아이템 ID와 설명을 담은 테이블
    '''
    cust_ind = np.where (customer_list == customer_id)[0][0]
    purchased_ind = mf_train[cust_ind,:].nonzero()[1]
    prod_codes = products_list[purchased_ind]
    
    return item_lookup[item_lookup.contentcode.isin(prod_codes.tolist())]

In [50]:
customers_arr = np.array(customers)
products_arr = np.array(products)

In [54]:
print(customers_arr[:5])

['aabkmf32@google.com' 'abploam6685@google.com' 'adcc3677@google.com'
 'adhbx60@google.com' 'adkrx@google.com']


In [55]:
get_items_purchased('aabkmf32@google.com', product_train, customers_arr, products_arr, item_lookup)

,contentcode,contentdes
1307,316104,보문사(서울)
1471,2396801,오랜미래신화미술관
1975,699587,천안 병천순대거리
2410,126372,능가사
2434,662234,순천전통야생차체험관
...,...,...
62309,2606232,호미반도 해안둘레길
77880,1599474,법인정사(김제)
77958,2611135,내원암 계곡
88595,134248,초가원가든


In [56]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

In [57]:
# pd.DataFrame(user_vecs[0,:].todense()) #1 X 20
# pd.DataFrame(item_vecs.todense()) # 20 X 995
# rec_vector = user_vecs[cust_ind,:].dot(item_vecs)
# pd.DataFrame(rec_vector.todense()) # 1 X 995

#pd.DataFrame(user_vecs.todense())

pref_vec = product_train[0,:].toarray() # Get the ratings from the training set ratings matrix
pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
rec_vector = user_vecs[0,:].dot(item_vecs) # Get dot product of user vector and all item vectors
rec_vector = rec_vector.toarray()

# Scale this recommendation vector between 0 and 1
min_max = MinMaxScaler()
rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 

recommend_vector = pref_vec*rec_vector_scaled
#print(recommend_vector)

product_idx = np.argsort(recommend_vector)[::-1][:5]
product_idx

array([23358, 22794, 12014, 23473, 23468])

In [58]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    유저의 추천 아이템 반환
    -----------------------------------------------------
    INPUT
    1. customer_id - Input the customer's id number that you want to get recommendations for
    2. mf_train: 훈련 데이터
    3. user_vecs: 행렬 분해에 쓰인 유저 벡터
    4. item_vecs: 행렬 분해에 쓰인 아이템 벡터
    5. customer_list: 평점 행렬의 행에 해당하는 고객 ID
    6. item_list: 평점 행렬의 열에 해당하는 아이템 ID
    7. item_lookup: 아이템 ID와 설명을 담은 테이블
    8. num_items: 추천할 아이템 개수
    -----------------------------------------------------
    반환    
    구매한 적이 없는 아이템 중 예측 평점이 높은 최고 n개의 추천 아이템
    '''
    
    pref_vec = product_train[0,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[0,:].dot(item_vecs) # Get dot product of user vector and all item vectors
    rec_vector = rec_vector.toarray()

    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 

    recommend_vector = pref_vec*rec_vector_scaled
    
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.contentdes.loc[item_lookup.contentcode == code].iloc[0]]) 
        # Append our descriptions to the list
        
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'contentcode': codes, 'contentdes': descriptions}) # Create a dataframe 
    return final_frame[['contentcode', 'contentdes']] # Switch order of columns around


### 'aabkmf32@google.com' 유저가 이미 클릭해본 contents

In [60]:
get_items_purchased('aabkmf32@google.com', product_train, customers_arr, products_arr, item_lookup)

,contentcode,contentdes
1307,316104,보문사(서울)
1471,2396801,오랜미래신화미술관
1975,699587,천안 병천순대거리
2410,126372,능가사
2434,662234,순천전통야생차체험관
...,...,...
62309,2606232,호미반도 해안둘레길
77880,1599474,법인정사(김제)
77958,2611135,내원암 계곡
88595,134248,초가원가든


### 'aabkmf32@google.com' 유저가 아직 클릭해본 적이 없으며, 추천할 만한 contents
 - 해당 리스트가 결국 화면에 나타나야 하는 것

In [61]:
rec_items('aabkmf32@google.com', product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup, num_items = 10)

,contentcode,contentdes
0,2589895,시간을 담다
1,1926273,미소레 커피
2,2460289,한국생명공학연구원
3,2581507,[착한가게] 장터수육
4,800214,"월남 파병용사 만남의 장 (구, 베트남 참전용사 만남의 장)"
5,1960045,청계서원(함안)
6,2668001,카페궁
7,404037,CK뉴욕바닷가재
8,822861,한강
9,127322,환벽당
